In [ ]:
def sync_recover(vct_ADCout, vct_ADCout_x_axis):

  mk = 0; mu = 0; NCO = 0; W_in = -1; integral_n = 0; position_prev = 0;
  vct_iter = np.array([]); vct_mu = np.array([]); vct_interp = np.array([]); vct_mk_plus_mu = np.array([]); 
  vct_error_n = np.array([]); vct_error_n_x = np.array([]); vct_W = np.array([]);

  for iter in np.arange(2, SYMBOLS*UPS_VALUE -2):

    vct_iter = np.append(vct_iter, iter);

    [mk, mu, NCO, W] = nco_0(W_in, iter, mk, mu, NCO) 
    vct_mu = np.append(vct_mu, mu);
    #mu_fixed = 1.00; mu = mu_fixed # Conmutador. Descomentar para prueba de error de fase a lazo abierto. Comentar para lazo cerrado.

    [sample_interp, sample_position] = interpol(vct_ADCout, mk, mu, vct_ADCout_x_axis[0])

    if (iter > 2) and (np.abs(sample_position - position_prev) > 1) : # last condition as Travis Trigger's TED
      #print('mk: {}, mu: {}'.format(mk, mu))
      #print('sample_interp: {}, sample_position: {}'.format(sample_interp, sample_position))
      vct_interp = np.append(vct_interp, sample_interp); vct_mk_plus_mu = np.append(vct_mk_plus_mu, sample_position);
      position_prev = sample_position; 
      
      if (len(vct_interp) >= (UPS_VALUE-1)) and (len(vct_interp)+1) % (UPS_VALUE//2) == 0: 
        
        error_n = ted( np.array([vct_interp[-(UPS_VALUE//2+1)], vct_interp[-(UPS_VALUE//4+1)], vct_interp[-1]]) )
        vct_error_n = np.append(vct_error_n, error_n); vct_error_n_x = np.append(vct_error_n_x, sample_position);
  
        [W_out, integral_n] = loop_filter(error_n, integral_n)
        vct_W = np.append(vct_W, W_out); 
        W_in = W_out # Conmutador. Comentar para prueba de error de frecuencia a lazo abierto. Descomentar para lazo cerrado.
  
  return [vct_iter, vct_mu, vct_interp, vct_mk_plus_mu, vct_error_n, vct_error_n_x, vct_W]

[![CC BY 4.0][cc-by-shield]][cc-by]

This work is licensed under a
[Creative Commons Attribution 4.0 International License][cc-by].

[![CC BY 4.0][cc-by-image]][cc-by]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-image]: https://i.creativecommons.org/l/by/4.0/88x31.png
[cc-by-shield]: https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg

![logo_neon_erasmus](https://raw.githubusercontent.com/neon-iot/communication_labs/main/detection_theory/notebooks/images/BannerSupportErasmus_.png)